In [6]:
import pandas as pd
import numpy as np

import spacy as sp
import regex as re

import tensorflow as tf    
from tensorflow import keras
import tensorflow_hub as hub

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_csv("./train.csv")

In [3]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
SYMBOL_FILTER = re.compile("[!@#%:;,\.?\'\"]")
CLEAN_UP = re.compile("[^a-z]")
WHITESPCAE_FILTER = re.compile("  *")

def extract_words(x):
    x = " ".join(SYMBOL_FILTER.sub("",x.lower()).split())
    x = " ".join(CLEAN_UP.sub(" ",x).split())
    x = " ".join(WHITESPCAE_FILTER.sub(" ",x).split())
    return x

df['comment_text_clean'] = df.comment_text.apply(extract_words)

In [8]:
cv = CountVectorizer()
features = cv.fit_transform(df.comment_text_clean)

In [9]:
tokenizer = keras.preprocessing.text.Tokenizer()

In [10]:
tokenizer.fit_on_texts(df.comment_text_clean)

In [11]:
tokens = tokenizer.texts_to_sequences(df.comment_text_clean)

In [12]:
tokens = keras.preprocessing.sequence.pad_sequences(tokens)

In [13]:
X,x,Y,y = train_test_split(tokens/1.0,df.toxic.values)

In [15]:
model = keras.Sequential([
    keras.layers.Dense(128, input_shape=(1403,)),
    keras.layers.Activation('relu'),
    keras.layers.Dense(256,),
    keras.layers.Activation('relu'),
    keras.layers.Dense(2),
    keras.layers.Activation('softmax'),
])


In [16]:
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"]
             )

In [17]:
model.fit(X,Y,epochs=10)

Train on 119678 samples
Epoch 1/10
119678/119678 [==============================] - 6s 54us/sample - loss: 29.0899 - acc: 0.8490
Epoch 2/10
119678/119678 [==============================] - 6s 52us/sample - loss: 0.4100 - acc: 0.9004
Epoch 3/10
119678/119678 [==============================] - 6s 52us/sample - loss: 0.3299 - acc: 0.9045
Epoch 4/10
119678/119678 [==============================] - 6s 52us/sample - loss: 0.3287 - acc: 0.9047
Epoch 5/10
119678/119678 [==============================] - 6s 53us/sample - loss: 0.3296 - acc: 0.9047
Epoch 6/10
119678/119678 [==============================] - 7s 56us/sample - loss: 0.3188 - acc: 0.9048
Epoch 7/10
119678/119678 [==============================] - 7s 55us/sample - loss: 0.3279 - acc: 0.9048
Epoch 8/10
119678/119678 [==============================] - 7s 56us/sample - loss: 0.3145 - acc: 0.9048
Epoch 9/10
119678/119678 [==============================] - 7s 62us/sample - loss: 0.3191 - acc: 0.9048
Epoch 10/10
119678/119678 [============

In [18]:
model.evaluate(x,y)

39893/39893 [==============================] - 1s 36us/sample - loss: 0.3204 - acc: 0.9022


[0.3204411357881025, 0.9022134]

In [19]:
pred = model.predict_classes(x)

In [37]:
from sklearn.metrics import precision_score,recall_score,accuracy_score,confusion_matrix

In [38]:
precision_score(y,pred),recall_score(y,pred),accuracy_score(pred,y)

(1.0, 0.00025627883136852895, 0.90221342090091)

In [39]:
confusion_matrix(y,pred)

array([[35991,     0],
       [ 3901,     1]])

39893